In [41]:
import numpy as np
import pandas as pd
import datetime
import math
from os import listdir
from os.path import isfile, join
import json

In [42]:
out='../output/'
inb='../input/flow/'
pref='idm'
inp = inb + pref + '/'

In [43]:
dat=pd.DataFrame(columns=['date','flow','from','hour','to'])
onlyfiles = [f for f in listdir(inp ) if isfile(join(inp, f))]
for f in onlyfiles:
    if (f.split('_')[0]==pref):
        #print(f)
        with open(join(inp, f), "r") as read_file:
            data = pd.DataFrame(json.load(read_file))
            dat=pd.concat([dat,data])

dat['flow']=dat['flow'].astype(int)
dat

,date,flow,from,hour,to
0,2021-04-01,94,MOZN_EDM,0,ZIMA
1,2021-04-01,94,ZIMA,0,BOT
2,2021-04-01,47,MOZN_EDM,1,ZIMA
3,2021-04-01,47,ZIMA,1,BOT
4,2021-04-01,56,MOZN_EDM,2,ZIMA
...,...,...,...,...,...
125,2023-06-28,6,ZAMC,19,ZAMZ
126,2023-06-28,6,ZAMZ,19,ZIM
127,2023-06-28,6,ZIM,19,ZIMA
128,2023-06-28,6,ZIMA,19,BOT


In [44]:
dam = pd.read_csv(out + pref + '_hourly.csv')
dam=dam.set_index([pd.DatetimeIndex(dam['date']),'hour'])
dam=dam.drop(['date'],1)
dam

C:\Users\kabom\AppData\Local\Temp\ipykernel_27836\1849715270.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  dam=dam.drop(['date'],1)


price  volume  year  month  day  day_of_week   value  cat
date       hour                                                           
2020-01-01 0      30.0    75.0  2020      1    1            2  2250.0  off
           1      30.0    70.0  2020      1    1            2  2100.0  off
           2       0.0     0.0  2020      1    1            2     0.0  off
           3      30.0    65.0  2020      1    1            2  1950.0  off
           4       0.0     0.0  2020      1    1            2     0.0  off
...                ...     ...   ...    ...  ...          ...     ...  ...
2023-07-02 19    150.0    35.0  2023      7    2            6  5250.0  off
           20    150.0    35.0  2023      7    2            6  5250.0  off
           21      0.0     0.0  2023      7    2            6     0.0  off
           22      0.0     0.0  2023      7    2            6     0.0  off
           23      0.0     0.0  2023      7    2            6     0.0  off

[24448 rows x 8 columns]

In [45]:
flow=dat
#flow['date']=pd.to_datetime(flow['flowdate'],format="%d/%m/%Y") #"%Y-%m-%d")#"%d/%m/%Y")
#flow=flow.drop(['flowdate','flowid','market'],axis=1)

flow=flow.set_index([pd.DatetimeIndex(flow['date'])])
flow['year']=flow.index.year
flow['month']=flow.index.month
flow['day']=flow.index.day
flow['day_of_week']=flow.index.dayofweek

flow=flow.set_index([pd.DatetimeIndex(flow['date']),'hour'])
flow['price']=dam['price']
flow['value']=flow['price']*flow['flow']
flow=flow.drop(['date'],axis=1)
flow=flow.dropna(0)
flow=flow.reset_index()

flow

C:\Users\kabom\AppData\Local\Temp\ipykernel_27836\2288514685.py:15: FutureWarning: In a future version of pandas all arguments of DataFrame.dropna will be keyword-only.
  flow=flow.dropna(0)


,date,hour,flow,from,to,year,month,day,day_of_week,price,value
0,2021-04-01,0,94,MOZN_EDM,ZIMA,2021,4,1,3,30.0,2820.0
1,2021-04-01,0,94,ZIMA,BOT,2021,4,1,3,30.0,2820.0
2,2021-04-01,1,47,MOZN_EDM,ZIMA,2021,4,1,3,30.0,1410.0
3,2021-04-01,1,47,ZIMA,BOT,2021,4,1,3,30.0,1410.0
4,2021-04-01,2,56,MOZN_EDM,ZIMA,2021,4,1,3,30.0,1680.0
...,...,...,...,...,...,...,...,...,...,...,...
16236,2023-06-28,19,6,ZAMC,ZAMZ,2023,6,28,2,120.0,720.0
16237,2023-06-28,19,6,ZAMZ,ZIM,2023,6,28,2,120.0,720.0
16238,2023-06-28,19,6,ZIM,ZIMA,2023,6,28,2,120.0,720.0
16239,2023-06-28,19,6,ZIMA,BOT,2023,6,28,2,120.0,720.0


In [46]:
f_to=flow[['date','hour','flow','value','price','to','year','month','day','day_of_week']].rename(columns={'to': 'node'})
f_from=flow[['date','hour','flow','value','price','from','year','month','day','day_of_week']].rename(columns={'from': 'node'})
f_from['flow']=-f_from['flow']
f_from['value']=-f_from['value']

flows=pd.concat([f_to, f_from])

flows

,date,hour,flow,value,price,node,year,month,day,day_of_week
0,2021-04-01,0,94,2820.0,30.0,ZIMA,2021,4,1,3
1,2021-04-01,0,94,2820.0,30.0,BOT,2021,4,1,3
2,2021-04-01,1,47,1410.0,30.0,ZIMA,2021,4,1,3
3,2021-04-01,1,47,1410.0,30.0,BOT,2021,4,1,3
4,2021-04-01,2,56,1680.0,30.0,ZIMA,2021,4,1,3
...,...,...,...,...,...,...,...,...,...,...
16236,2023-06-28,19,-6,-720.0,120.0,ZAMC,2023,6,28,2
16237,2023-06-28,19,-6,-720.0,120.0,ZAMZ,2023,6,28,2
16238,2023-06-28,19,-6,-720.0,120.0,ZIM,2023,6,28,2
16239,2023-06-28,19,-6,-720.0,120.0,ZIMA,2023,6,28,2


In [47]:
flows.loc[flows['node']=='ZIMA','node']='ZIM'
flows.loc[flows['node']=='MOZN_EDM','node']='MOZ'
flows.loc[flows['node']=='MOZN_HCB','node']='HCB'
flows.loc[flows['node']=='MOZS','node']='MOZ'
flows.loc[flows['node']=='RSAS','node']='RSA'
flows.loc[flows['node']=='RSAN','node']='RSA'

In [48]:
flows=flows.reset_index().groupby(['date','hour','node','year','month','day','day_of_week']).sum().reset_index()
flows=flows.loc[flows['flow']!=0]
flows=flows.set_index(['date','hour'])
flows['cat']=dam['cat']
flows.reset_index()
flows

node  year  month  day  day_of_week  index  flow   value  \
date       hour                                                             
2021-04-01 0      BOT  2021      4    1            3      1    94  2820.0   
           0      MOZ  2021      4    1            3      0   -94 -2820.0   
           1      BOT  2021      4    1            3      3    47  1410.0   
           1      MOZ  2021      4    1            3      2   -47 -1410.0   
           2      BOT  2021      4    1            3      5    56  1680.0   
...               ...   ...    ...  ...          ...    ...   ...     ...   
2023-06-28 18    ZAMC  2023      6   28            2  16232    -6 -1020.0   
           19     BOT  2023      6   28            2  16239     6   720.0   
           19    ZAMC  2023      6   28            2  16236    -6  -720.0   
           21     ESW  2023      6   28            2  16240    44  1320.0   
           21     MOZ  2023      6   28            2  16240   -44 -1320.0   

                 price       cat  
date       hour                   
2021-04-01 0      30.0       off  
           0      30.0       off  
           1      30.0       off  
           1      30.0       off  
           2      30.0       off  
...                ...       ...  
2023-06-28 18    170.0   evening  
           19    120.0   evening  
           19    120.0   evening  
           21     30.0  standard  
           21     30.0  standard  

[9838 rows x 10 columns]

In [49]:
flowsb=flows.copy()
flowsb.loc[flowsb['flow']<0,'side']='supply'
flowsb.loc[flowsb['flow']>0,'side']='consume'


flow_monthly_cat=flowsb.groupby(['year','month', 'node','cat','side']).sum()
flow_monthly_cat.rename(columns={'flow': 'flow_sum'}, inplace=True)
flow_monthly_cat['hour_count']=flowsb.groupby(['year','month', 'node','cat','side']).count()['flow']
flow_monthly_cat['price_weight_mean'] = flow_monthly_cat['value'] / flow_monthly_cat['flow_sum']
flow_monthly_cat['price_mean']=flowsb.groupby(['year','month', 'node','cat','side']).mean()['price']
flow_monthly_cat=flow_monthly_cat.drop(['day','day_of_week','index','price'],axis=1)
flow_monthly_cat=flow_monthly_cat.loc[flow_monthly_cat['flow_sum']!=0]
flow_monthly_cat['power']=flow_monthly_cat['flow_sum']/flow_monthly_cat['hour_count']
flow_monthly_cat

flow_sum      value  hour_count  \
year month node cat      side                                       
2021 4     BOT  evening  consume       137   13879.68           3   
                morning  consume      1178  105948.13          25   
                off      consume      2236   77258.43          51   
                standard consume      4816  319730.29          83   
           ESW  off      consume       186    6231.60           6   
...                                    ...        ...         ...   
2023 6     MOZ  morning  supply       -171   -6657.00          10   
                off      supply       -206   -3145.00          12   
                standard supply       -954  -30329.00          51   
           ZAMC evening  supply        -20   -2740.00           4   
           ZAMZ evening  supply        -54   -6366.00           2   

                                  price_weight_mean  price_mean      power  
year month node cat      side                                               
2021 4     BOT  evening  consume         101.311533  140.813333  45.666667  
                morning  consume          89.938990   89.113600  47.120000  
                off      consume          34.552071   35.630784  43.843137  
                standard consume          66.389180   75.195301  58.024096  
           ESW  off      consume          33.503226   33.493333  31.000000  
...                                             ...         ...        ...  
2023 6     MOZ  morning  supply           38.929825   37.700000 -17.100000  
                off      supply           15.266990   15.500000 -17.166667  
                standard supply           31.791405   32.980392 -18.705882  
           ZAMC evening  supply          137.000000  135.000000  -5.000000  
           ZAMZ evening  supply          117.888889  124.500000 -27.000000  

[449 rows x 6 columns]

In [50]:
flow_monthly_cat.to_csv(out + pref + '_flow_monthly_cat.csv')